# Automated ML

Importing Depedencies required for the project

In [6]:
import os
import pandas as pd
import numpy as np
import json
import requests
import joblib
from sklearn.metrics import confusion_matrix
import itertools

from azureml.core import Dataset, Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.widgets import RunDetails
from azureml.train.automl import AutoMLConfig

from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.data.dataset_factory import TabularDatasetFactory

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'Auto-stoke'

experiment=Experiment(ws, experiment_name)

In [9]:
found = False
key = 'strokeDataset'
description_text = "Prediction of Stroke"

if key in ws.datasets.keys():
    found = True
    dataset = ws.datasets[key]
    
if not found :
    example = 'https://raw.githubusercontent.com/123manju900/Capstone-AzureML/main/stroke-prediction-dataset.csv'
    dataset = Dataset.Tabular.from_delimited_files(example)


    dataset = dataset.register(workspace = ws,
                          name = key , 
                          description = description_text )



In [3]:
from train2 import clean_data

In [12]:
df = dataset.to_pandas_dataframe()

In [14]:
dataset.take(5).to_pandas_dataframe()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,True,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,True,Self-employed,Rural,202.21,N/A,never smoked,1
2,31112,Male,80.0,0,1,True,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,True,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,True,Self-employed,Rural,174.12,24,never smoked,1


In [15]:
data_train, data_test = dataset.random_split(0.7)

## Configuring Compute Cluster 

In [16]:
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D12_V2',
                                                           max_nodes=5)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration



In [21]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 4,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 6
}
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=data_train,
                             label_column_name="stroke", 
                             enable_early_stopping= True,
                             featurization= 'auto',
                             **automl_settings
                            )

In [22]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on cpu-cluster with default configuration
Running on remote compute: cpu-cluster


Experiment,Id,Type,Status,Details Page,Docs Page
Auto-stoke,AutoML_73aa99f8-b9da-425e-8ddf-8cb0a8ce7a33,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+--------------------------------------+
|Size of the smallest class       |Name/Label of the smallest class |Number of samples in the training data|
+=================================+==========================

## Run Details


In [23]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
Auto-stoke,AutoML_73aa99f8-b9da-425e-8ddf-8cb0a8ce7a33,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [25]:
RunDetails(remote_run).show

<bound method _WidgetRunDetailsBase.show of <azureml.widgets._automl._run_details._AutoMLRunDetails object at 0x7f3819ba02e8>>

In [26]:
remote_run.wait_for_completion()

{'runId': 'AutoML_73aa99f8-b9da-425e-8ddf-8cb0a8ce7a33',
 'target': 'cpu-cluster',
 'status': 'Completed',
 'startTimeUtc': '2021-04-18T22:12:10.971159Z',
 'endTimeUtc': '2021-04-18T22:38:27.653508Z',
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '6',
  'target': 'cpu-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Auto-stoke","subscription_id":"b968fb36-f06a-4c76-a15f-afab68ae7667","resource_group":"aml-quickstarts-143086","workspace_name":"quick-starts-ws-143086","region":"southcentralus","compute_target":"cpu-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"iterations":1000,"primary_metric":"accuracy","task_type":"classification","data_script":null,"test_size":0.0,"validation_size":0.0,"n

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [27]:
best_run, fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics()

In [28]:
best_run

Experiment,Id,Type,Status,Details Page,Docs Page
Auto-stoke,AutoML_73aa99f8-b9da-425e-8ddf-8cb0a8ce7a33_36,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [29]:
fitted_model


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                subsample_for_bin=200000,
                                                                                                subsample_freq=0,
                                                                                                verbose=-10))],
                                            

In [30]:
print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print(fitted_model._final_estimator)
print(best_run.get_tags())

Best Run Id:  AutoML_73aa99f8-b9da-425e-8ddf-8cb0a8ce7a33_36

 Accuracy: 0.7913830680216468
PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('15',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            LightGBMClassifier(boosting_type='goss',
                                                                               class_weight=None,
                                                                               colsample_bytree=0.3966666666666666,
                                                                               importance_type='split',
                                                                         

In [31]:
os.makedirs('./outputs', exist_ok=True)

joblib.dump(fitted_model, filename='outputs/automl.joblib')

model_name = best_run.properties['model_name']
model_name

'AutoML73aa99f8b36'

In [36]:
from azureml.automl.core.shared import constants 
env = best_run.get_environment()

script_file = 'score.py'

best_run.download_file('outputs/scoring_file_v_1_0_0.py', script_file)
best_run.download_file(constants.CONDA_ENV_FILE_PATH, 'env.yml')

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [43]:
#Register the fitted model
model = remote_run.register_model(model_name = model_name,
                                  description = 'AutoML_model')

In [45]:
inference_config = InferenceConfig(entry_script = script_file, environment = env)

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                memory_gb = 1, 
                                                enable_app_insights = True,
                                                auth_enabled = True
                                               description = 'Predict whether person had stroke or not')
                                            

aci_service_name = 'automl-webservice1'
print(aci_service_name)

automl-webservice1


In [46]:
service = Model.deploy(ws, aci_service_name, [model], inference_config, aci_config)
service.wait_for_deployment(True)
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-18 23:26:00+00:00 Creating Container Registry if not exists.
2021-04-18 23:26:00+00:00 Registering the environment.
2021-04-18 23:26:01+00:00 Use the existing image.
2021-04-18 23:26:01+00:00 Generating deployment configuration.
2021-04-18 23:26:03+00:00 Submitting deployment to compute..
2021-04-18 23:26:07+00:00 Checking the status of deployment automl-webservice1..
2021-04-18 23:30:48+00:00 Checking the status of inference endpoint automl-webservice1.
Succeeded
ACI service creation operation finished, operation "Succeeded"
State: Healthy
Scoring URI: http://e4f662e9-12fa-46c1-83c9-e1ef5c48beb1.southcentralus.azurecontainer.io/score


In [47]:
print("State: " + service.state)
print("Scoring URI: " + service.scoring_uri)
print("Keys: " + service.get_keys()[0])
print("Swagger URI: " + service.swagger_uri)

State: Healthy
Scoring URI: http://e4f662e9-12fa-46c1-83c9-e1ef5c48beb1.southcentralus.azurecontainer.io/score
Keys: HzlqTWKDLKghgXcPVAOp3gkzA4ahAauQ
Swagger URI: http://e4f662e9-12fa-46c1-83c9-e1ef5c48beb1.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [51]:
data_test = data_test.dropna()
data_sample = data_test.sample(3)
y_true = data_sample.pop('stroke')
sample_json = json.dumps({'data':data_sample.to_dict(orient='records')})
print(sample_json)

{"data": [{"id": 47844, "gender": "Female", "age": 38.0, "hypertension": 0, "heart_disease": 0, "ever_married": "Yes", "work_type": "Private", "Residence_type": "Urban", "avg_glucose_level": 69.34, "bmi": 43.7, "smoking_status": "never smoked"}, {"id": 57618, "gender": "Female", "age": 47.0, "hypertension": 0, "heart_disease": 0, "ever_married": "Yes", "work_type": "Self-employed", "Residence_type": "Rural", "avg_glucose_level": 140.39, "bmi": 25.5, "smoking_status": "never smoked"}, {"id": 56575, "gender": "Female", "age": 51.0, "hypertension": 1, "heart_disease": 0, "ever_married": "Yes", "work_type": "Govt_job", "Residence_type": "Urban", "avg_glucose_level": 69.94, "bmi": 33.3, "smoking_status": "smokes"}]}


In [50]:
data_test = pd.read_csv('https://raw.githubusercontent.com/123manju900/Capstone-AzureML/main/stroke-prediction-dataset.csv')

In [52]:
service.get_logs(())

'2021-04-18T23:30:25,356665500+00:00 - rsyslog/run \n2021-04-18T23:30:25,359412100+00:00 - iot-server/run \n2021-04-18T23:30:25,366626700+00:00 - gunicorn/run \n2021-04-18T23:30:25,377213200+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_8e5a5a51349877e7d47c6a2872e0ebfd/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)

TODO: In the cell below, print the logs of the web service and delete the service

In [ ]:
service.delete()